In [5]:
import numpy as np
import os
import glog as log
import copy

from __future__ import division

import open3d as o3d
from open3d import JVisualizer
import pandas as pd

from evaluation.tools.mesh import Mesh
from evaluation.tools.mesh_evaluator import MeshEvaluator

# Rotation matrices:
# East North Up (ENU) frame to Unity's world frame of reference
enu_R_unity = np.array([[1, 0, 0],
                        [0, 0, 1],
                        [0, 1, 0]])
unity_R_enu = np.transpose(enu_R_unity)

# Right Handed frame to Unity's Left Handed frame of reference
righthand_R_lefthand = np.array([[1, 0, 0],
                                 [0, -1, 0],
                                 [0, 0, 1]])
lefthand_R_righthand = np.transpose(righthand_R_lefthand)

In [2]:
# FILL PATHS BELOW
#gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
#est_mesh_path = "/home/tonirv/Downloads/tesse_semantics_2.ply"

#gt_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/semantic_mesh_tonirv_ld_9118_6487309760727328010.ply"

#gt_mesh_path = "/home/tonirv/Downloads/tesse_multiscene_office1_3d_semantic_v5.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/tesse_semantics_3.ply"

gt_mesh_path = "/home/tonirv/Downloads/office1_toni.ply"
est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/office2_v3.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/office2_v3_sparkvio.ply"
#est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/office2_v3_sparkvio_dense_stereo.ply"

gt_mesh_path = "/home/tonirv/datasets/euroc/EuRoC/V1_01_easy/mav0/pointcloud0/data.ply"
est_mesh_path = "/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/mesh_results/euroc_v1_02_spark_vio.ply"

In [3]:
print("Loading Ground-truth mesh...")
gt_mesh_original = Mesh(gt_mesh_path)
print("Loading Estimated mesh...")
est_mesh_original = Mesh(est_mesh_path)

Loading Ground-truth mesh...


NameError: name 'Mesh' is not defined

In [8]:
# Transform Meshes to same frame of reference
gt_mesh = copy.deepcopy(gt_mesh_original)
est_mesh = copy.deepcopy(est_mesh_original)

In [9]:
# Align Pointclouds Manually:
#est_mesh.mesh_o3d.translate([0, -5, 0])
#gt_mesh.transform_left(righthand_R_lefthand)
gt_mesh.transform_left(enu_R_unity)
#o3d.io.write_triangle_mesh("office1_toni_enu.ply", gt_mesh.mesh_o3d)

Transforming mesh according to left matrix:
[[1 0 0]
 [0 0 1]
 [0 1 0]]


In [10]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(est_mesh.mesh_o3d)
vis.add_geometry(gt_mesh.mesh_o3d)
vis.add_geometry(o3d.geometry.create_mesh_coordinate_frame(size=4))
vis.run()
vis.destroy_window()

In [11]:
NUMBER_OF_SAMPLES=1000000
gt_pcl = o3d.geometry.sample_points_uniformly(gt_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
# Don't sample estimated mesh, just pick vertices, otw you'll be mixing colors...
# est_pcl = o3d.geometry.sample_points_uniformly(est_mesh.mesh_o3d, NUMBER_OF_SAMPLES)
est_pcl = o3d.io.read_point_cloud(est_mesh_path)

In [12]:
# Calculate normals for nice visualization
# THIS COLORS THE PCL?>>>>>????
#o3d.geometry.estimate_normals(
#        est_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))
#o3d.geometry.estimate_normals(
#        gt_pcl,
#        search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1,
#                                                          max_nn=5))

In [13]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().mesh_show_back_face = True
vis.add_geometry(gt_pcl)
vis.add_geometry(est_pcl)
vis.run()
vis.destroy_window()

In [14]:
# ICP
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
def draw_correspondences(source, target, correspondences):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    #source_temp.paint_uniform_color([1, 0.706, 0])
    #target_temp.paint_uniform_color([0, 0.651, 0.929])
    o3d.visualization.draw_geometries([source_temp, #target_temp, 
                                       correspondences])
    

In [15]:
# ICP params
ICP_THRESHOLD = 1.5
import math
yaw = 0
tx = 0
ty = 0
tz = 0
if False:
    yaw = 44
    tx = 18.5
    ty = 25
    tz = 2.5
yaw_rad = math.radians(yaw)
trans_init = np.asarray([[math.cos(yaw_rad), -math.sin(yaw_rad), 0.0, tx],
                         [math.sin(yaw_rad), math.cos(yaw_rad), 0.0, ty],
                         [0.0, 0.0, 1.0, tz],
                         [0.0, 0.0, 0.0, 1.0]])


In [16]:
# Visualize initial registration problem
draw_registration_result(est_pcl, gt_pcl, trans_init)

In [17]:
# Evaluate current fit between pointclouds
evaluation = o3d.registration.evaluate_registration(est_pcl, gt_pcl, ICP_THRESHOLD, trans_init)

In [18]:
print("Initial registration")
print(evaluation)

Initial registration
registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.099000, and correspondence_set size of 475061
Access transformation to get result.


In [19]:
print("Apply point-to-point ICP")
reg_p2p = o3d.registration.registration_icp(
    est_pcl, gt_pcl, ICP_THRESHOLD, trans_init,
    o3d.registration.TransformationEstimationPointToPoint(),
    o3d.registration.ICPConvergenceCriteria(max_iteration = 2000))
correspondences = reg_p2p.correspondence_set

Apply point-to-point ICP


In [20]:
print(reg_p2p)
print("")

print("Transformation is:")
print(reg_p2p.transformation)
print("")

print("Correspondence Set:")
print(reg_p2p.correspondence_set)
print("")

registration::RegistrationResult with fitness = 1.000000, inlier_rmse = 0.079332, and correspondence_set size of 475061
Access transformation to get result.

Transformation is:
[[ 0.99999515  0.00157796 -0.00268335  0.08755623]
 [-0.00157206  0.99999634  0.00220284  0.07571541]
 [ 0.00268681 -0.00219861  0.99999397  0.04083394]
 [ 0.          0.          0.          1.        ]]

Correspondence Set:
std::vector<Eigen::Vector2i> with 475061 elements.
Use numpy.asarray() to access data.



In [21]:
# Apply tf to 3D mesh and write:
est_mesh_icped = copy.deepcopy(est_mesh.mesh_o3d).transform(reg_p2p.transformation)
o3d.visualization.draw_geometries([est_mesh_icped])
o3d.io.write_triangle_mesh("office2_v3_icped.ply", est_mesh_icped)

True

In [22]:
# Draw Registration Result
draw_registration_result(est_pcl, gt_pcl, reg_p2p.transformation)

In [23]:
# Draw Only Correspondences
c2c_lines = o3d.geometry.create_line_set_from_point_cloud_correspondences(est_pcl, gt_pcl, correspondences)
o3d.visualization.draw_geometries([c2c_lines])

In [24]:
# Draw PointClouds and Correspondences
draw_correspondences(est_pcl, gt_pcl, c2c_lines)

In [25]:
def calc_corresp(est_pcl, gt_pcl, correspondences):
    total_negative_matches = 0
    total_positive_matches = 0
    total_correspondences = len(correspondences)
    for correspondence in correspondences:
        if np.allclose(est_pcl.colors[correspondence[0]],
                       gt_pcl.colors[correspondence[1]]):
            total_positive_matches += 1
        else:
            total_negative_matches += 1

    print("Positive color matches: ",total_positive_matches)
    print("Negative color matches: ", total_negative_matches)
    print("Total correspondences: ", total_correspondences)
    assert(total_correspondences == total_negative_matches + total_positive_matches)
    print ("Positive: {}  % ".format(total_positive_matches / total_correspondences * 100))
    print ("Negative: {}  % ".format(total_negative_matches / total_correspondences * 100))

In [26]:
calc_corresp(est_pcl, gt_pcl, correspondences)

('Positive color matches: ', 425453)
('Negative color matches: ', 49608)
('Total correspondences: ', 475061)
Positive: 89.5575515565  % 
Negative: 10.4424484435  % 


In [27]:
# Import Semantic Labels
df = pd.read_csv('/home/tonirv/Code/ROS/flight_goggles_ws/src/voxblox/voxblox_ros/cfg/tesse_multiscene_office1_segmentation_mapping.csv')

In [ ]:
normalized_df = copy.deepcopy(df)
normalized_df['normalized_red'] = df['red'] / 255
normalized_df['normalized_green'] = df['green'] / 255
normalized_df['normalized_blue'] = df['blue'] / 255
#print(sum(np.isclose(normalized_df['normalized_red'], gt_pcl.colors[0][0])))

In [ ]:
from hashlib import sha1

from numpy import all, array, uint8


class hashable(object):
    r'''Hashable wrapper for ndarray objects.
        Instances of ndarray are not hashable, meaning they cannot be added to
        sets, nor used as keys in dictionaries. This is by design - ndarray
        objects are mutable, and therefore cannot reliably implement the
        __hash__() method.
        The hashable class allows a way around this limitation. It implements
        the required methods for hashable objects in terms of an encapsulated
        ndarray object. This can be either a copied instance (which is safer)
        or the original object (which requires the user to be careful enough
        not to modify it).
    '''
    def __init__(self, wrapped, tight=False):
        r'''Creates a new hashable object encapsulating an ndarray.
            wrapped
                The wrapped ndarray.
            tight
                Optional. If True, a copy of the input ndaray is created.
                Defaults to False.
        '''
        self.__tight = tight
        self.__wrapped = array(wrapped) if tight else wrapped
        self.__hash = int(sha1(wrapped.view(uint8)).hexdigest(), 16)

    def __eq__(self, other):
        return all(self.__wrapped == other.__wrapped)

    def __hash__(self):
        return self.__hash

    def unwrap(self):
        r'''Returns the encapsulated ndarray.
            If the wrapper is "tight", a copy of the encapsulated ndarray is
            returned. Otherwise, the encapsulated ndarray itself is returned.
        '''
        if self.__tight:
            return array(self.__wrapped)



In [ ]:
# No need to hash, we have ids now
red_col = df['red'].to_numpy()
blue_col = df['blue'].to_numpy()
green_col = df['green'].to_numpy()

hash_factor = 10
def f(x):    
    return str(int(x['normalized_red']*hash_factor))+str(int(x['normalized_green']*hash_factor))+str(int(x['normalized_blue']*hash_factor))

hashed_df = copy.deepcopy(normalized_df)
hashed_df['hash'] = hashed_df.apply(f, axis=1)
hashed_df

In [ ]:
# Generate table from color to id.
def label_from_hash(hash):
    return hashed_df.loc[(hashed_df['hash'] == hash)].iat[0, 5]
def hash_from_color(color):
    r = int(color[0]*hash_factor)
    g = int(color[1]*hash_factor)
    b = int(color[2]*hash_factor)
    return str(r)+str(g)+str(b)

In [ ]:
hash_from_color([0.007843,0.403922,0.278431])

In [ ]:
hash_to_label_dict = {i: label_from_hash(i) for i in hashed_df['hash'].unique().tolist()}
print hash_to_label_dict

In [ ]:
# Compare labels between correspondences:
unique_ids = hashed_df['id'].unique().tolist()
unique_ids.sort()
print unique_ids
confusion_matrix = {i:{j:0 for j in unique_ids} for i in unique_ids}

In [ ]:
def calc_per_label_corresp():
    for correspondence in correspondences:
        est_pcl_color = est_pcl.colors[correspondence[0]]
        gt_pcl_color  = gt_pcl.colors[correspondence[1]]
        try:
            est_label_id = hash_to_label_dict[hash_from_color(est_pcl_color)]
            gt_label_id = hash_to_label_dict[hash_from_color(gt_pcl_color)]
        except:
            print hash_from_color(gt_pcl_color)
            #print "Est: ", est_pcl_color
            #print "GT: ", gt_pcl_color
            
        confusion_matrix[est_label_id][gt_label_id] += 1

In [ ]:
calc_per_label_corresp()

In [ ]:

print hashed_df.loc[(hashed_df['id'] == 0)]

In [ ]:
# Generate data for heatmap:
def label_id_to_name(id):
    if id == 0:
        return r'$\text{Unknown}$'
    elif id == 1:
        return r'$\text{Airvent.}$'
    elif id == 2:
        return r'$\text{Book}$'
    elif id == 3:
        return r'$\text{Floor}$'
    elif id == 4:
        return r'$\text{Ceil.}$'
    elif id == 5:
        return r'$\text{Chair}$'
    elif id == 6:
        return ""
    elif id == 7:
        return r'$\text{Couch}$'
    elif id == 8:
        return ""
    elif id == 9:
        return r'$\text{Shelf}$'
    elif id == 10:
        return r'$\text{Fan}$'
    elif id == 11:
        return r'$\text{Lamp}$'
    elif id == 12:
        return r'$\text{Paint.}$'
    elif id == 13:
        return r'$\text{Plant}$'
    elif id == 14:
        return r'$\text{Sign}$'
    elif id == 15:
        return r'$\text{Stairs}$'
    elif id == 16:
        return r'$\text{Table}$'
    elif id == 17:
        return r'$\text{Screen}$'
    elif id == 18:
        return r'$\text{Bin}$'
    elif id == 19:
        return r'$\text{Wall}$'
    
remove_keys = [0, 2, 4, 13, 14, 15]        
x = [label_id_to_name(id) for id in unique_ids if id not in remove_keys ]
y = x
z = []
for est_key, gt in confusion_matrix.items():
    if est_key not in remove_keys:
        values = []
        for gt_key, value in gt.items():
            if gt_key not in remove_keys:
                values.append(value)
        z.append(values)
print x
print y
print z

In [ ]:
import plotly.graph_objects as go
from plotly.graph_objs import Data
trace1 = {
  "type": "heatmap", 
  "x": x, 
  "y": y, 
  "zmax": 10000, 
  "zmin": 0, 
  "z": z, 
  "colorscale": [[0, "rgb(255,245,240)"], [0.2, "rgb(254,224,210)"], [0.4, "rgb(252,187,161)"],  [0.5, "rgb(252,146,114)"], [0.6, "rgb(251,106,74)"],   [0.7, "rgb(239,59,44)"],  [0.8, "rgb(203,24,29)"], [0.9, "rgb(165,15,21)"], [1, "rgb(103,0,13)"]],
  "autocolorscale": False
}
data = Data([trace1])

layout = {
  "title": r'$\text{Confusion Matrix}$', 
  "width": 400,
  "height": 400,
  "xaxis": {
    "title": r'$\text{Predicted}$',
    "titlefont": {
      "size": 18
    }
  }, 
  "yaxis": {
    "title": r'$\text{Ground-Truth}$', 
    "titlefont": {
      "size": 18
    }
  }
}

fig = go.Figure(data=data, layout=layout)
fig.show()

In [ ]:
# Calculate IoU:
# TODO(Toni): Ignore Unknown!
np_confusion_matrix = np.matrix(z)
TP = np.diag(np_confusion_matrix)
FP_TP = np.sum(np_confusion_matrix, axis=0, dtype = 'int')
FP = np.squeeze(np.asarray(FP_TP)) - TP
FN_TP = np.sum(np_confusion_matrix, axis=1)
FN = np.squeeze(np.asarray(FN_TP)) - TP
TN = np.sum(np_confusion_matrix) - TP - FP - FN
print "TP:\n", TP
print "FP:\n", FP
print "FN:\n", FN
print "TN:\n", TN
#iou = cm.diag() / (cm.sum(dim=1) + cm.sum(dim=0) - cm.diag() + 1e-15)
IoU = TP / (np.squeeze(np.asarray(FP_TP)) + np.squeeze(np.asarray(FN_TP)) - TP + 1e-15)
print "IoU:\n", IoU
mIoU = np.mean(IoU)
print "mIoU:\n", mIoU
mAcc = np.sum(TP) / np.sum(np_confusion_matrix)
print "mAcc:\n", mAcc

In [ ]:
# Setup Orca for figure generation:
import plotly.io as pio
import plotly
# Install orca using the AppImage (if you are in Ubuntu)
plotly.io.orca.config.executable = '/usr/local/bin/orca'
# Print orca config to check: pio.orca.config


In [ ]:
# Save figures to images:
if not os.path.exists("images"):
    os.mkdir("images")
#fig.write_image("/home/tonirv/Documents/Research/Papers/2019w-IROS-SparkVIO/img/confusion_matrix.svg")
#fig.write_image("/home/tonirv/Documents/Research/Papers/2019w-IROS-SparkVIO/img/confusion_matrix.png")
#fig.write_image("/home/tonirv/Documents/Research/Papers/2019w-IROS-SparkVIO/img/confusion_matrix.eps")